To do:
- Make first_note and last_note work
- Fix 9/11/sus2/sus4 detection
- Fix chord_changes
- Make melodies work

In [100]:
import pretty_midi as pm
import matplotlib.pyplot as plt
# Env variables
chrom_notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B'] # A list of all the notes/pitch classes with
                                                                                # indices corresponding to 
                                                                                # MIDI note values mod 12

offsets = { # A list of chord intervals with their corresponding MIDI note value offset
    '1': 0, 
    '2': 2,
    '3': 4,
    '4': 5,
    '5': 7,
    '6': 9,
    '7': 11,
    '8': 12,
    '9': 14,
    '10': 16,
    '11': 17,
    '12': 19,
    '13': 21
}

In [101]:
# Returns the first note by time in a list of notes
def first_note(notes):
    if notes == []:
        return None
    f_note = notes[0]
    for i in range(1, len(notes)):
        if notes[i].start < f_note.start:
            f_note = notes[i]
    return f_note

# Returns the last note by time in a list of notes
def last_note(notes):
    if notes == []:
        return None
    l_note = notes[0]
    for i in range(1, len(notes)):
        if notes[i].start > l_note.start:
            l_note = notes[i]
    return l_note

# Returns a list of all the non-drum notes in a song regardless of instrument
def consolidate_notes(song):
    notes = []
    for instrument in song.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                notes.append(note)
    return notes

# Returns a note name based on its MIDI note number
def get_note(note_n):
    return chrom_notes[note_n % 12]

# Returns the note corresponding to a particular degree in a scale defined by the root note
def parse_chord(root, number_string):
    note_num = chrom_notes.index(root)
    out = ""
    num = ""
    scale_num = 0
    parentheses = False
    for char in number_string:
        if char == '(':
            parentheses = True
        if char == 'b':
            scale_num -= 1
        if char == '#':
            scale_num += 1
        if char >= '0' and char <= '9':
            num += char
    scale_num += offsets.get(num)
    if (parentheses):
        out = "("
    out += str(get_note(note_num + scale_num))
    if (parentheses):
        out += ")"
    return out

# Outputs the master chord list (dict version)
def generate_chord_list(filepath = "chords without names.txt"):
    chord_list = []
    for note in chrom_notes:
        f = open(filepath)
        lines = f.readlines()
        for line in lines:
            parts = line.split()
            chord_name = ''
            note_list = []
            for i in range(len(parts)):
                part = parts[i]
                if i == 0:
                    chord_name = part.replace('_', note, 1)
                elif part[0] == 'b' or part[0] == '#' or \
                   (part[0] >= '0' and part[0] <= '9') or \
                   part[0] == '(':
                    note_list.append(parse_chord(note, part))
                else: continue
            chord_list.append([chord_name, note_list])
    return chord_list

# Gets the chords in a song
def get_chords(notes, 
               offset = 0.01):
    start_times = []
    for note in notes:
        if not (note.start in start_times):
            start_times.append(note.start)
    chords = []
    for time in start_times:
        playing_notes = []
        actual = time + offset # Offset is a parameter shifting the time selected for to allow chords to be picked up
        for note in notes:
            if note.start < actual and note.end >= time:
                song.instruments[0].notes
                playing_notes.append(note)
        chords.append(playing_notes)
    return chords

# Gets the chords in a song
def get_chords_window(notes, 
                      offset = 0.01,
                      window = 0.5):
    start_times = []
    for note in notes:
        if not (note.start in start_times):
            start_times.append(note.start)
    chords = []
    for time in start_times:
        playing_notes = []
        # Offset is a parameter shifting the time selected for to allow chords to be picked up
        # Window is a parameter allowing notes behind the current to be picked up
        for note in notes:
            if note.start < time + offset and note.end >= time - window:
                song.instruments[0].notes
                playing_notes.append(note)
        chords.append(playing_notes)
    return chords

# Generates note scores for the piece
def get_note_scores(notes, 
                    octave_multiplier_on = False,
                    end_multiplier_on = False):
    note_scores_octave_agn = []
    note_scores_octave_agn_dict = dict()
    last_start = last_note(notes).start
    first_start = first_note(notes).start
    last_end = last_note(notes).start
    overall_dur = last_end - first_start
    overall_dur_minus_last = last_start - first_start
    for i in range(0, 12):
        note_scores_octave_agn.append(0) # Create bins for each note
    for note in notes:
        duration = note.end - note.start
        score = duration * note.velocity / 127
        octave_multiplier = 1
        end_multiplier = 1
        if octave_multiplier_on: # Reduce the score of the note the higher up in pitch it is
            octave_multiplier = max(0, 1 - (max(0, (round(note.pitch / 12) - 2) / 8.0)))
        if end_multiplier_on: # Reduce the score of the note the farther away it is from the last note
            end_multiplier = (note.start - first_start) / overall_dur_minus_last
        score *= octave_multiplier
        score *= end_multiplier
        note_scores_octave_agn[note.pitch % 12] += score # Add the note scores by pitch class
    for i in range(0, 12):
        if note_scores_octave_agn[i] != 0:
            note_scores_octave_agn_dict[i] = note_scores_octave_agn[i]

    return note_scores_octave_agn_dict, overall_dur, last_end, first_start
    
# Generates chord scores based on note scores
def get_chord_scores(chord_list, 
                     note_scores_octave_agn_dict, 
                     overall_dur,
                     parentheses_multiplier = 1,
                     min_note_threshold = 0.1, 
                     missing_deweight = 0.5, 
                     root_note_multiplier = 2):
    chord_scores_dict = {}
    for chord_tuple in chord_list:
        chord_name = chord_tuple[0]
        chord_notes = chord_tuple[1]
        chord_score = 0.0
        for i in range(0, len(chord_notes)):
            note = chord_notes[i]
            multiplier = 1 # A multiplier for the note score when calculating chord matchups
            actual_note = note
            if note[0] == '(':
                multiplier = parentheses_multiplier
                actual_note = note[1 : (len(note) - 1)]
            if i == 0: # If the note is the root note, weight that pitch specifically
                multiplier *= root_note_multiplier
            note_val = chrom_notes.index(actual_note)
            note_score = note_scores_octave_agn_dict.get(note_val, 0) # Grab the actual note score
            if note_score <= min_note_threshold: # Deweight chords with missing notes
                note_score = -1 * missing_deweight
            chord_score += note_score * multiplier # Multiply by the multiplier and sum to the chord score
        if chord_score > 0.0:
            chord_scores_dict[chord_name] = chord_score
    chord_scores_dict_sorted = sorted(chord_scores_dict.items(), key=lambda x:x[1], reverse = True) # Sort the chords
                                                                                                    # by score
    return chord_scores_dict_sorted

# Makes a list of all the chords in a song
def calculate_song_chords(song):
    all_chords = generate_chord_list()
    chord_list = []
    for chord in get_chords(song.instruments[0].notes):
        note_dict, overall_dur, last_end, first_start = get_note_scores(chord)
        
        chord_scores = get_chord_scores(all_chords, note_dict, last_end)
        if chord_scores != []:
            chord = chord_scores[:1][0][0] # Grab the top detected chord for each chord event
            if chord != "":
                chord_list.append(chord)
    return chord_list

# Returns n-grams of the items of a list (used in this case to make chord-grams)
def n_grams(my_list, n):
    items = []
    for i in range(0, len(my_list) - n):
        n_gram = []
        for j in range(i, i + n):
            n_gram.append(my_list[j])
        items.append(n_gram)
    return items

# Returns the number of chord changes per second on average of a song
def chord_changes(chord_list, song):
    notes = song.instruments[0].notes
    duration = last_note(notes).end - first_note(notes).start
    chord_changes_per_time = (len(chord_list) - 1) / duration
    return chord_changes_per_time

In [102]:
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\weird.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\Space Theme.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\gen hoshino-Piano.mid")
song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav.midi")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\progression.mid")
# song = pm.PrettyMIDI("C:\\Users\\TPNml\\Downloads\\Untitled score.mid")

In [103]:
all_chords = generate_chord_list()
notes = consolidate_notes(song)
for chord in get_chords(notes):
    note_dict, overall_dur, last_end, first_start = get_note_scores(chord, octave_multiplier_on = True)
    # print(note_dict)
    print(get_chord_scores(all_chords, note_dict, last_end)[:1], end = " @ ")
    print(f"%.2f s" % first_start)

[('Gb5', 0.060147022637795367)] @ 1.05 s
[('Gb5', 0.7616726131889765)] @ 1.05 s
[] @ 1.62 s
[] @ 2.30 s
[] @ 2.51 s
[] @ 2.51 s
[] @ 2.93 s
[('Gb5', 0.46422859251968507)] @ 2.93 s
[] @ 3.21 s
[] @ 3.62 s
[('Gb5', 2.132617700131233)] @ 3.62 s
[('Gb5', 1.5217868274278215)] @ 3.77 s
[('Gb5', 1.005823490813648)] @ 3.77 s
[('Gb5', 1.005823490813648)] @ 3.77 s
[('Gb5', 1.005823490813648)] @ 3.77 s
[('Gb5', 1.1167979002624666)] @ 3.77 s
[('Gb5', 1.254962270341207)] @ 3.77 s
[('Gb5', 1.254962270341207)] @ 3.77 s
[('Gb5', 1.005823490813648)] @ 3.77 s
[('Gb5', 2.132617700131233)] @ 3.62 s
[('Gb5', 1.608702171505905)] @ 3.77 s
[('DM7', 1.5333581959481617)] @ 5.92 s
[('D', 1.5812546136811014)] @ 6.08 s
[('D5', 1.2245939960629912)] @ 6.08 s
[('D5', 1.2245939960629912)] @ 6.08 s
[('D5', 1.394685039370078)] @ 6.08 s
[('D5', 1.394685039370078)] @ 6.08 s
[('D', 1.3651421013779517)] @ 6.08 s
[('D', 1.5711890994094477)] @ 6.08 s
[('D5', 1.4306409940944873)] @ 6.08 s
[('D5', 1.2245939960629912)] @ 5.92 s


[('Asus2', 1.2941452386811032)] @ 53.69 s
[('Asus2', 0.6913139763779541)] @ 53.69 s
[('A5', 0.4778312623031503)] @ 54.27 s
[] @ 54.27 s
[] @ 54.80 s
[('Dbaug', 0.7709461122047226)] @ 55.15 s
[('Dbaug', 0.7555671751968487)] @ 55.15 s
[('Dbaug', 0.7555671751968487)] @ 55.15 s
[('Dbaug', 0.7951294906496045)] @ 55.15 s
[('Db5', 0.5570404773622035)] @ 54.94 s
[('Db5', 0.5570404773622035)] @ 54.94 s
[('Db5', 0.5570404773622035)] @ 55.15 s
[] @ 55.16 s
[] @ 56.51 s
[] @ 56.70 s
[] @ 56.92 s
[] @ 57.03 s
[('A5', 0.19177534448819067)] @ 57.03 s
[] @ 57.03 s
[] @ 57.17 s
[] @ 57.70 s
[] @ 57.89 s
[] @ 57.89 s
[] @ 58.26 s
[] @ 58.43 s
[] @ 58.43 s
[] @ 58.61 s
[] @ 58.64 s
[] @ 58.61 s
[('Do', 1.3253234703083983)] @ 59.14 s
[('Do', 1.3253234703083983)] @ 59.14 s
[('D5', 0.3919834727690269)] @ 58.64 s
[('Do', 1.3253234703083983)] @ 59.14 s
[('Do', 0.8182876066272946)] @ 58.64 s
[('Abo', 0.748195538057742)] @ 59.14 s
[('Ab5', 0.3526082677165354)] @ 59.18 s
[] @ 60.54 s
[] @ 60.72 s
[] @ 60.72 s
[]

[] @ 113.26 s
[] @ 113.36 s
[] @ 113.36 s
[] @ 113.55 s
[] @ 113.55 s
[] @ 113.56 s
[] @ 113.56 s
[] @ 113.64 s
[] @ 113.64 s
[] @ 113.67 s
[] @ 113.67 s
[] @ 113.67 s
[] @ 113.67 s
[] @ 113.67 s
[] @ 113.64 s
[] @ 113.67 s
[] @ 113.67 s
[] @ 114.11 s
[] @ 114.12 s
[] @ 114.20 s
[] @ 114.20 s
[] @ 114.20 s
[] @ 114.20 s
[] @ 114.20 s
[] @ 114.20 s
[] @ 114.11 s
[] @ 114.20 s
[] @ 114.20 s
[('Db5', 7.9718257874015706)] @ 115.16 s
[('Dbsus4', 9.315903871391068)] @ 115.16 s
[('Db5', 5.119463582677166)] @ 115.16 s
[] @ 124.64 s
[] @ 124.89 s
[] @ 125.13 s
[] @ 125.39 s
[] @ 125.64 s
[] @ 125.95 s
[] @ 126.44 s
[('Db5', 0.775928887795279)] @ 126.60 s
[('Db5', 0.775928887795279)] @ 126.60 s
[('Db5', 0.775928887795279)] @ 126.60 s
[('Db5', 0.775928887795279)] @ 126.60 s
[('Db5', 0.6541789698162788)] @ 126.60 s
[('Dbaug', 0.9424058809055166)] @ 126.60 s
[('Dbaug', 0.9424058809055166)] @ 126.60 s
[('Dbaug', 0.9424058809055166)] @ 126.60 s
[('Db5', 0.8162729658792709)] @ 126.60 s
[('Db5', 0.6541

[] @ 168.18 s
[] @ 168.18 s
[] @ 168.18 s
[] @ 168.20 s
[] @ 168.28 s
[] @ 168.28 s
[] @ 168.28 s
[] @ 168.41 s
[] @ 168.41 s
[] @ 168.41 s
[] @ 168.67 s
[] @ 168.67 s
[] @ 168.67 s
[] @ 168.67 s
[] @ 168.91 s
[] @ 168.91 s
[] @ 168.91 s
[] @ 168.91 s
[] @ 169.15 s
[] @ 169.15 s
[] @ 169.15 s
[] @ 169.15 s
[] @ 169.39 s
[] @ 169.39 s
[] @ 169.39 s
[] @ 169.39 s
[] @ 169.63 s
[] @ 169.63 s
[] @ 169.63 s
[] @ 169.63 s
[('Db5', 0.7688289452099579)] @ 169.94 s
[('Db5', 0.7688289452099579)] @ 169.94 s
[('Db5', 0.42972338418633593)] @ 169.94 s
[('Db5', 0.7688289452099579)] @ 169.94 s
[('Db5', 0.7688289452099579)] @ 169.94 s
[('Db5', 0.08964895013121787)] @ 169.94 s
[] @ 170.35 s
[] @ 170.35 s
[] @ 170.35 s
[] @ 170.35 s
[] @ 170.35 s
[] @ 170.35 s
[] @ 170.35 s
[] @ 170.35 s
[] @ 170.98 s
[] @ 170.98 s
[] @ 170.98 s
[] @ 170.98 s
[] @ 171.12 s
[] @ 171.12 s
[] @ 171.12 s
[] @ 171.31 s
[] @ 171.31 s
[] @ 171.31 s
[] @ 171.31 s
[] @ 171.31 s
[] @ 171.64 s
[] @ 171.64 s
[] @ 171.64 s
[] @ 171.6

[] @ 212.55 s
[] @ 213.55 s
[] @ 213.55 s
[] @ 213.55 s
[] @ 213.55 s
[] @ 213.55 s
[] @ 213.74 s
[] @ 213.55 s
[] @ 213.55 s
[] @ 213.74 s
[] @ 213.74 s
[] @ 213.74 s
[] @ 213.74 s
[] @ 214.50 s
[] @ 214.50 s
[] @ 214.50 s
[] @ 214.50 s
[] @ 214.50 s
[] @ 214.50 s
[] @ 214.66 s
[] @ 214.50 s
[] @ 214.82 s
[] @ 215.09 s
[] @ 215.09 s
[] @ 215.10 s
[] @ 215.10 s
[] @ 215.10 s
[] @ 215.24 s
[] @ 215.24 s
[] @ 215.24 s
[] @ 215.76 s
[] @ 215.76 s
[] @ 215.76 s
[] @ 215.91 s
[] @ 215.91 s
[] @ 215.91 s
[] @ 216.09 s
[] @ 216.09 s
[] @ 216.09 s
[] @ 216.40 s
[] @ 216.40 s
[] @ 216.53 s
[] @ 216.53 s
[] @ 216.75 s
[] @ 216.75 s
[] @ 216.75 s
[] @ 216.75 s
[] @ 217.09 s
[] @ 217.09 s
[] @ 217.09 s
[] @ 217.23 s
[] @ 217.23 s
[] @ 217.23 s
[] @ 217.45 s
[] @ 217.45 s
[] @ 217.45 s
[] @ 217.45 s
[] @ 217.79 s
[] @ 217.79 s
[] @ 217.79 s
[] @ 217.99 s
[] @ 217.99 s
[] @ 217.99 s
[] @ 218.29 s
[('Abm', 0.0903435654527559)] @ 218.29 s
[('Abm', 0.0903435654527559)] @ 218.29 s
[] @ 218.29 s
[] @ 218

...

[] @ 1575.36 s
[] @ 1575.33 s
[] @ 1575.36 s
[] @ 1576.19 s
[] @ 1576.30 s
[] @ 1576.30 s
[] @ 1576.30 s
[] @ 1576.30 s
[] @ 1576.30 s
[] @ 1576.30 s
[] @ 1576.30 s
[] @ 1576.82 s
[] @ 1576.30 s
[] @ 1577.38 s
[] @ 1577.54 s
[] @ 1577.54 s
[] @ 1577.55 s
[] @ 1577.83 s
[] @ 1577.83 s
[] @ 1577.83 s
[] @ 1578.32 s
[] @ 1578.32 s
[] @ 1578.18 s
[] @ 1578.32 s
[] @ 1578.63 s
[] @ 1578.78 s
[] @ 1579.06 s
[] @ 1579.06 s
[] @ 1579.21 s
[] @ 1579.57 s
[] @ 1579.57 s
[] @ 1579.68 s
[] @ 1579.70 s
[] @ 1579.83 s
[] @ 1580.17 s
[] @ 1580.11 s
[] @ 1580.35 s
[] @ 1580.17 s
[] @ 1580.78 s
[] @ 1580.78 s
[] @ 1580.78 s
[] @ 1580.92 s
[] @ 1581.12 s
[] @ 1581.12 s
[] @ 1581.41 s
[] @ 1581.41 s
[] @ 1581.27 s
[] @ 1581.41 s
[] @ 1581.74 s
[] @ 1582.14 s
[] @ 1581.74 s
[] @ 1582.14 s
[] @ 1582.14 s
[] @ 1582.29 s
[] @ 1582.67 s
[] @ 1582.45 s
[] @ 1582.96 s
[] @ 1582.96 s
[] @ 1583.35 s
[] @ 1583.35 s
[] @ 1583.35 s
[] @ 1583.90 s
[] @ 1583.90 s
[] @ 1583.90 s
[] @ 1583.90 s
[] @ 1584.15 s
[] @ 1584.

[] @ 1628.43 s
[] @ 1628.53 s
[] @ 1628.43 s
[] @ 1628.53 s
[] @ 1628.53 s
[] @ 1628.53 s
[] @ 1628.65 s
[] @ 1628.65 s
[] @ 1628.94 s
[] @ 1629.02 s
[] @ 1629.02 s
[] @ 1629.02 s
[] @ 1629.02 s
[] @ 1629.02 s
[] @ 1629.14 s
[] @ 1629.35 s
[] @ 1629.35 s
[] @ 1629.35 s
[] @ 1629.35 s
[] @ 1629.35 s
[] @ 1629.56 s
[] @ 1629.56 s
[] @ 1629.56 s
[] @ 1629.56 s
[] @ 1629.56 s
[] @ 1629.56 s
[] @ 1629.45 s
[] @ 1629.56 s
[] @ 1629.93 s
[] @ 1630.00 s
[] @ 1629.56 s
[] @ 1630.00 s
[] @ 1630.10 s
[] @ 1630.23 s
[] @ 1630.10 s
[] @ 1630.23 s
[] @ 1630.33 s
[] @ 1630.23 s
[] @ 1630.44 s
[] @ 1630.44 s
[] @ 1630.44 s
[] @ 1630.34 s
[] @ 1630.67 s
[] @ 1630.67 s
[] @ 1630.44 s
[] @ 1630.67 s
[] @ 1630.76 s
[] @ 1630.76 s
[] @ 1630.99 s
[] @ 1630.89 s
[] @ 1630.99 s
[] @ 1631.12 s
[] @ 1631.23 s
[] @ 1631.23 s
[] @ 1631.23 s
[] @ 1631.60 s
[] @ 1631.60 s
[] @ 1631.69 s
[] @ 1631.69 s
[] @ 1631.69 s
[] @ 1631.69 s
[] @ 1631.80 s
[] @ 1631.94 s
[] @ 1632.02 s
[] @ 1632.02 s
[] @ 1632.02 s
[] @ 1632.

[] @ 1691.25 s
[] @ 1691.49 s
[] @ 1691.76 s
[] @ 1691.76 s
[] @ 1691.89 s
[] @ 1691.89 s
[] @ 1692.18 s
[] @ 1692.18 s
[] @ 1692.32 s
[] @ 1692.54 s
[] @ 1692.54 s
[] @ 1692.54 s
[] @ 1692.54 s
[] @ 1692.79 s
[] @ 1692.83 s
[] @ 1692.79 s
[] @ 1692.79 s
[] @ 1692.83 s
[] @ 1693.03 s
[] @ 1693.09 s
[] @ 1693.09 s
[] @ 1693.09 s
[] @ 1693.03 s
[] @ 1693.09 s
[] @ 1693.35 s
[] @ 1693.35 s
[] @ 1693.35 s
[] @ 1693.35 s
[] @ 1693.60 s
[] @ 1693.60 s
[] @ 1693.60 s
[] @ 1693.60 s
[] @ 1693.60 s
[] @ 1693.83 s
[] @ 1693.83 s
[] @ 1693.83 s
[] @ 1693.83 s
[] @ 1693.83 s
[] @ 1693.85 s
[] @ 1694.06 s
[] @ 1694.12 s
[] @ 1694.12 s
[] @ 1694.06 s
[] @ 1694.06 s
[] @ 1694.06 s
[] @ 1694.12 s
[] @ 1694.48 s
[] @ 1694.56 s
[] @ 1694.56 s
[] @ 1694.56 s
[] @ 1694.56 s
[] @ 1694.82 s
[] @ 1695.27 s
[] @ 1695.42 s
[] @ 1695.51 s
[] @ 1695.51 s
[] @ 1695.67 s
[] @ 1695.80 s
[] @ 1695.94 s
[] @ 1695.94 s
[] @ 1696.05 s
[] @ 1696.17 s
[] @ 1696.30 s
[] @ 1696.35 s
[] @ 1696.30 s
[] @ 1696.30 s
[] @ 1696.

[] @ 1728.10 s
[] @ 1728.10 s
[] @ 1728.49 s
[] @ 1728.49 s
[] @ 1728.49 s
[] @ 1728.49 s
[] @ 1728.69 s
[] @ 1728.69 s
[] @ 1728.69 s
[] @ 1728.90 s
[] @ 1728.90 s
[] @ 1728.90 s
[] @ 1728.90 s
[] @ 1728.90 s
[] @ 1728.90 s
[] @ 1728.90 s
[] @ 1729.29 s
[] @ 1729.29 s
[] @ 1729.29 s
[] @ 1729.29 s
[] @ 1729.49 s
[] @ 1729.49 s
[] @ 1729.49 s
[] @ 1729.76 s
[] @ 1729.76 s
[] @ 1729.76 s
[] @ 1729.76 s
[] @ 1729.76 s
[] @ 1729.77 s
[] @ 1729.77 s
[] @ 1729.77 s
[] @ 1729.77 s
[] @ 1730.39 s
[] @ 1730.39 s
[] @ 1730.39 s
[] @ 1730.39 s
[] @ 1730.59 s
[] @ 1730.59 s
[] @ 1730.59 s
[] @ 1730.77 s
[] @ 1730.77 s
[] @ 1730.77 s
[] @ 1730.77 s
[] @ 1730.98 s
[] @ 1730.98 s
[] @ 1730.98 s
[] @ 1731.16 s
[] @ 1731.16 s
[] @ 1731.16 s
[] @ 1731.35 s
[] @ 1731.35 s
[] @ 1731.35 s
[] @ 1731.35 s
[] @ 1731.53 s
[] @ 1731.53 s
[] @ 1731.53 s
[] @ 1731.53 s
[] @ 1731.71 s
[] @ 1731.71 s
[] @ 1731.71 s
[] @ 1731.71 s
[] @ 1731.90 s
[] @ 1731.90 s
[] @ 1731.90 s
[] @ 1731.90 s
[] @ 1732.06 s
[] @ 1732.

[] @ 1760.62 s
[] @ 1760.95 s
[] @ 1760.95 s
[] @ 1760.95 s
[] @ 1760.96 s
[] @ 1760.95 s
[] @ 1761.35 s
[] @ 1761.35 s
[] @ 1761.35 s
[] @ 1761.35 s
[('Fo', 1.725687182168562)] @ 1761.95 s
[('Fo', 1.1109795357610817)] @ 1761.95 s
[('Fo', 1.725687182168562)] @ 1761.95 s
[('Fo', 0.9385611466534901)] @ 1761.95 s
[('Db7', 1.7381595000820038)] @ 1761.95 s
[('F5', 0.6236774114172696)] @ 1761.95 s
[('F5', 0.6236774114172696)] @ 1761.95 s
[('F5', 0.6236774114172696)] @ 1761.95 s
[] @ 1763.43 s
[] @ 1763.43 s
[] @ 1763.43 s
[] @ 1763.43 s
[('Ebo', 0.9485113188977936)] @ 1763.86 s
[('Ebo', 0.9485113188977936)] @ 1763.86 s
[('Ao', 0.7148745078741662)] @ 1763.86 s
[('Ebo', 1.7421772473754487)] @ 1763.86 s
[('Ebo', 1.7421772473754487)] @ 1763.86 s
[('A5', 0.4002214566930101)] @ 1763.86 s
[] @ 1765.20 s
[('A5', 0.4002214566930101)] @ 1763.86 s
[] @ 1765.20 s
[] @ 1765.20 s
[('A5', 0.4002214566930101)] @ 1763.86 s
[('Bo', 0.40494150877628393)] @ 1765.55 s
[('Bo', 1.0746660207513816)] @ 1765.55 s
[('

In [ ]:
chords = calculate_song_chords(song)
print(chord_changes(chords, song))